In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from pylightgbm.models import GBMClassifier

_seed=49

def transformToXy(data):
    # Extract the data
    X = data.drop(['TARGET'], axis=1)
    # Extract the labels
    y = data['TARGET']
    return X, y

Regular dataset has around 300k rows with around 3% positives of the binary target class. It consists of regular numeric columns, ordered categoricals encoded as 1,2,3 and factor variables which are dummy-coded. Only the ones which do not have a too large number of factors are used as factors for the dummy coding. The other ones are transformed into a numeric space e.g. by calculating a percentage of the target class contained in this factor variable. In total (after the dummy coding) there are a little bit less than 10k columns.

Below I tried to generate a sample df, Howver did not (yet) succed.

In [9]:
targetdf = pd.DataFrame(np.random.randint(0,2,(100000,1)),columns=['TARGET'])
XDF = pd.DataFrame(np.random.randint(0,100,(100000,10)),columns=list('ABCDEFGHIJ'))
XDF['A'] = XDF['A'].astype('category')
XDF['B'] = XDF['B'].astype('category')
XDF['C'] = XDF['C'].astype('category')
XDF['D'] = XDF['D'].astype('category')

mydf = pd.concat([targetdf, XDF], join='inner', axis=1)
mydf = pd.get_dummies(mydf, sparse=False)
X, y = transformToXy(mydf)
X.shape

(100000, 406)

In [12]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=_seed)
pathToLightGBM = "/path/to/lightgbm"
for train_index, test_index in split.split(X, y):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    
    # minimum example does not yet work
    #estimator = GBMClassifier(exec_path=pathToLightGBM, 
    #                          num_iterations=4000, learning_rate=0.01,
    #                          num_leaves=2000, min_data_in_leaf=100, metric='binary_logloss',
    #                          feature_fraction=0.7, bagging_fraction=0.7, bagging_freq=0,
    #                          metric_freq=1, early_stopping_round=20,
    #                         tree_learner="feature", num_threads=4,
    #                          is_unbalance=True)
    # even here a strange failure
    estimator = GBMClassifier(exec_path=pathToLightGBM)#, 
                              #tree_learner="feature", num_threads=4,
                              #is_unbalance=True)
    
    estimator.fit(X_train, y_train)
    y_predicted = estimator.predict(X_test)
    print(y_predicted)

[LightGBM] [Info] Loading parameters .. finished
[LightGBM] [Info] Finish loading data, use 0.843833 seconds
[LightGBM] [Info] Number of postive:40023,  number of negative:39977
[LightGBM] [Info] Number of data:80000, Number of features:406
[LightGBM] [Info] Finish training initilization.
[LightGBM] [Info] Start train
[LightGBM] [Info] 0.033051 seconds elapsed, finished 1 iteration
[LightGBM] [Info] 0.074876 seconds elapsed, finished 2 iteration
[LightGBM] [Info] 0.108249 seconds elapsed, finished 3 iteration
[LightGBM] [Info] 0.146607 seconds elapsed, finished 4 iteration
[LightGBM] [Info] 0.182826 seconds elapsed, finished 5 iteration
[LightGBM] [Info] 0.216487 seconds elapsed, finished 6 iteration
[LightGBM] [Info] 0.250089 seconds elapsed, finished 7 iteration
[LightGBM] [Info] 0.281025 seconds elapsed, finished 8 iteration
[LightGBM] [Info] 0.306693 seconds elapsed, finished 9 iteration
[LightGBM] [Info] 0.336431 seconds elapsed, finished 10 iteration
[LightGBM] [Info] Finish trai

CalledProcessError: Command '['/Users/geoHeil/neverpayer/lightgbm', 'config=/var/folders/zm/bts0g37j0l9637sjxb3b700h0000gn/T/tmpelcap5qd/predict.conf']' returned non-zero exit status 1